Toy - CGE

*Lorang and Gerlagh 2022*

#### Packages import

In [35]:
import numpy as np
import pandas as pd
import copy
# import csv
from scipy.optimize import minimize
from copy import deepcopy
import matplotlib.pyplot as plt
# import matplotlib.backends.backend_pdf

# import random


# Model structure

### Data classes

In [36]:
# IMPORT AND READ A SAM MATRIX
# gives a description of the SAM, with number of sectors in total, per relevant subcategories (eg firms), and lists of sectors names

class SAM:
    def __init__(self, SAM_xls, SAM_sheet):
        self.sectors = pd.read_excel(SAM_xls,sheet_name=SAM_sheet, header=1, index_col=None, usecols='A:B')
        self.data = pd.read_excel(SAM_xls,sheet_name=SAM_sheet, header=1, index_col=1).drop(['Unnamed: 0'], axis=1)
        self.data.fillna(0., inplace=True)
        self.sectors.columns = ['category','sectors']
        self.sectorsNb = len(self.sectors.index)-1
        self.firmsStart = self.sectors[self.sectors['category'] == 'Firms'].index[0]
        self.factorsStart = self.sectors[self.sectors['category'] == 'Factors'].index[0]
        self.consumersStart = self.sectors[self.sectors['category'] == 'Consumers'].index[0]
        self.institutionsStart = self.sectors[self.sectors['category'] == 'Institutions'].index[0]
        self.materials = pd.read_excel(SAM_xls,sheet_name='MATERIALS',usecols='A')
        self.materialsNb = len(self.materials.index)

    def get_firmsNb(self):
        i=self.firmsStart+1
        while pd.isnull(self.sectors['category'].iloc[i])==True:
            i=i+1
        return i-self.firmsStart

    def get_factorsNb(self):
        i=self.factorsStart+1
        while pd.isnull(self.sectors['category'].iloc[i])==True:
            i=i+1
        return i-self.factorsStart

    def get_consumersNb(self):
        i=self.consumersStart+1
        while pd.isnull(self.sectors['category'].iloc[i])==True:
            i=i+1
        return i-self.consumersStart

    def get_institutionsNb(self):
        i=self.institutionsStart+1
        while pd.isnull(self.sectors['category'].iloc[i])==True:
            i=i+1
        return i-self.institutionsStart

    def get_firms(self):
        firms = self.sectors['sectors'].iloc[self.firmsStart:self.firmsStart+self.get_firmsNb()].values
        return firms

    def get_factors(self):
        factors = self.sectors['sectors'].iloc[self.factorsStart:self.factorsStart+self.get_factorsNb()].values
        return factors

    def get_consumers(self):
        hh = self.sectors['sectors'].iloc[self.consumersStart:self.consumersStart+self.get_consumersNb()].values
        return hh

    def get_institutions(self):
        hh = self.sectors['sectors'].iloc[self.institutionsStart:self.institutionsStart+self.get_institutionsNb()].values
        return hh

    def get_capitalNb(self):
        capitalNb = self.get_factors().tolist().index('CAPITAL')
        return capitalNb

    def get_investorNb(self):
        investorNb = self.get_consumers().tolist().index('INVESTOR')
        return investorNb

    def get_materials(self):
        mat = self.materials["Material"].values
        return mat

class PIOT:
    def __init__(self, PIOT_xls, PIOT_sheet, name='material'):
        # should be the same structure than SAM, but we want to check just in case by comparing both sectors columns
        self.name = name
        self.sectors = pd.read_excel(PIOT_xls,sheet_name=PIOT_sheet, header=1, index_col=None, usecols='A:B') 
        self.data = pd.read_excel(PIOT_xls,sheet_name=PIOT_sheet, header=1, index_col=1).drop(['Unnamed: 0'], axis=1)
        self.data.fillna(0., inplace=True)
        self.sectors.columns = ['category','sectors']
        self.sectorsNb = len(self.sectors.index)-2
        self.firmsStart = self.sectors[self.sectors['category'] == 'Firms'].index[0]
        self.factorsStart = self.sectors[self.sectors['category'] == 'Factors'].index[0]
        self.consumersStart = self.sectors[self.sectors['category'] == 'Consumers'].index[0]
        self.institutionsStart = self.sectors[self.sectors['category'] == 'Institutions'].index[0]

class sectors:
    def __init__(self,SAM):
        self.firms = SAM.get_firms()
        self.factors = SAM.get_factors()
        self.consumers = SAM.get_consumers()
        self.institutions = SAM.get_institutions()
        self.materials = SAM.get_materials()
    
    def get_sectors(self):
        list_sectors = np.concatenate((self.firms,self.factors,self.consumers,self.institutions))
        return list_sectors

    def get_abbr_sectors(self, length=3):
        list_abbr_sectors = [i[0:length] for i in self.get_sectors()]
        return list_abbr_sectors

class fixedParameters:
    def __init__(self,SAM):
        self.I = SAM.get_firmsNb()
        self.F = SAM.get_factorsNb()
        self.H = SAM.get_consumersNb()
        self.G = SAM.get_institutionsNb()
        self.capital = SAM.get_capitalNb()
        self.investor = SAM.get_investorNb()
        self.M = SAM.materialsNb

class calibParameters:
    def __init__(self,fP):
        self.alpha = np.full((fP.I,fP.I), np.nan, float)
        self.beta = np.full((fP.F,fP.I), np.nan, float)
        self.gamma = np.full((fP.I,fP.H), np.nan, float)
        self.eta = np.full((fP.I,fP.G), np.nan, float)
        self.Ai = np.full(fP.I, np.nan, float)
        self.Ah = np.full(fP.H, np.nan, float)
        self.Ag = np.full(fP.G, np.nan, float)
        self.Omega = np.full((fP.H-1,fP.F-1), np.nan, float) # for now we assume labor supply inelastic across households and factors, 
        self.sigma = np.full(fP.H, np.nan, float)
        # parameters for the material balance:
        self.epsilon = np.full((fP.M,fP.I), np.nan, float)
        self.rho = np.full((fP.M,fP.I), np.nan, float)
        self.bl = np.full((fP.M,fP.I), False, bool) # ARRAY OF LAMBDAS

class Parameters: #for parameters that are not calibrated with the SAM, e g capital degradation rate delta
    def __init__(self, fP):
        self.delta = np.full(1, np.nan, float)
        self.TFPgrowth = np.full(fP.I, np.nan, float)
        self.nbYear = np.full(1, np.nan, float)
        self.deltam = np.full(fP.M, np.nan, float)

class flowVariables:
    def __init__(self,fP):
        self.X = np.full((fP.I,fP.I), np.nan, float)
        self.L = np.full((fP.F,fP.I), np.nan, float)
        self.CH = np.full((fP.I,fP.H), np.nan, float)
        self.CG = np.full((fP.I,fP.G), np.nan, float)
        self.Omega = np.full((fP.H-1,fP.F), np.nan, float)
        self.TH = np.full((fP.G,fP.H), np.nan, float)
        self.TI = np.full((fP.G,fP.I), np.nan, float)
        self.TG = np.full((fP.G,fP.G), np.nan, float)
        self.S = np.full(fP.H-1, np.nan, float)
        self.Y = np.full(fP.I, np.nan, float)

class flowVariables_mat:
    def __init__(self,fP):
        self.Xm = np.full((fP.M,fP.I,fP.I), np.nan, float)
        self.CHm = np.full((fP.M,fP.I,fP.H), np.nan, float)
        self.CGm = np.full((fP.M,fP.I,fP.G), np.nan, float)
        self.Ym = np.full((fP.M,fP.I), np.nan, float)
        self.WIm = np.full((fP.M,fP.I), np.nan, float)
        self.WHm = np.full((fP.M,fP.H-1), np.nan, float)
        self.WGm = np.full((fP.M,fP.G), np.nan, float)
        self.Rm = np.full((fP.M,fP.I), np.nan, float)
        self.DeltaPSm = np.full(fP.M, np.nan, float) # inflow of capital
        self.DeltaMSm = np.full(fP.M, np.nan, float) # outflow of capital
        self.WKm = np.full((fP.M,fP.I), np.nan, float) # waste from capital outflow

class intensityVariables:
    def __init__(self,fP):
        self.thetaX = np.full((fP.M,fP.I,fP.I), np.nan, float)
        self.thetaCH = np.full((fP.M,fP.I,fP.H), np.nan, float)
        self.thetaCG = np.full((fP.M,fP.I,fP.G), np.nan, float)
        # self.thetaY = np.full((fP.M,fP.I), np.nan, float)

class stockVariables_mat:
    def __init__(self,fP):
        self.Km = np.full(fP.M, np.nan, float)  # I define a capital stock variable, basically, all material consumed by the investor goes into this. We could define intensity of capital (globally), or capital by firm. Note useful for now, but to keep in mind.

class controlVariables:
    def __init__(self,fP):
        self.y = np.full(fP.I, np.nan, float)
        self.zh = np.full(fP.H, np.nan, float)
        self.zg = np.full(fP.G, np.nan, float)
        self.pi = np.full(fP.I, np.nan, float)
        self.pf = np.full(fP.F, np.nan, float)

class controlVariables_mat:
    def __init__(self,fP):
        self.lbd = np.full((fP.M,fP.I), np.nan, float)

class arrayLambda:
    def __init__(self,fP):
        self.bl = np.full((fP.M,fP.I), False, bool) 

class policyVariables:
    def __init__(self,fP):
        self.tauH = np.full(fP.H-1, np.nan, float) # tax on consumers consumption (VAT)
        self.tauG = np.full(fP.G, np.nan, float) # VAT for institutions
        self.tauX = np.full(fP.I, np.nan, float) # tax on purchases of inputs and final goods (rates per good i are the same for H/G/I)
        self.tauR = np.full((fP.M,fP.I), np.nan, float) # tax on resource extraction
        self.tauWI = np.full((fP.M,fP.I),np.nan, float) # tax on waste at firm level
        self.tauWH = np.full((fP.M,fP.H-1),np.nan, float) # general tax on waste
        self.tauWG = np.full((fP.M,fP.G),np.nan, float) # institutions tax on waste

class reportVariables: #to be completed
    def __init__(self,fP,fV_mat):
        self.totalW = sum(fV_mat.WIm[:,i] for i in range(fP.I)) + sum(fV_mat.WHm[:,h] for h in range(fP.H-1)) + sum(fV_mat.WGm[:,g] for g in range(fP.G)) + fV_mat.DeltaMSm

class scenario:
    def __init__(self,fP,name="name", pV=None, cV=None, fV=None, cV_mat=None, fV_mat=None, iV=None, sV_mat=None, rV=None):
        self.name = name
        if pV is None:
            self.pV = policyVariables(fP)
        else:
            self.pV = pV
        if cV is None:
            self.cV = controlVariables(fP)
        else:
            self.cV = cV
        if fV is None:
            self.fV = flowVariables(fP)
        else:
            self.fV = fV
        if cV_mat is None:
            self.cV_mat = controlVariables_mat(fP)
        else:
            self.cV_mat = cV_mat
        if fV_mat is None:
            self.fV_mat = flowVariables_mat(fP)
        else:
            self.fV_mat = fV_mat
        if iV is None:
            self.iV = intensityVariables(fP)
        else:
            self.iV = iV
        if sV_mat is None:
            self.sV_mat = intensityVariables(fP)
        else:
            self.sV_mat = sV_mat
        if rV is None:
            self.rV = reportVariables(fP,self.fV_mat)
        else:
            self.rV = rV



### Exporting functions

In [37]:

# from xml.etree.ElementTree import tostring

def writeNewPIOT(sec, fV_mat, cV_mat, material): #new PIOT
    m = np.where(sec.materials == material)[0][0] #get the number of the material we want the piot of
    newPIOT = pd.DataFrame(columns = np.concatenate((sec.firms,sec.factors,sec.consumers,sec.institutions,["WASTE"],["STOCK"])), index = np.concatenate((sec.firms,sec.factors,sec.consumers,sec.institutions,["EXTRACTION"])))
    newPIOT.loc[sec.firms,sec.firms] = fV_mat.Xm[m,:,:]
    newPIOT.loc[sec.firms,sec.consumers] = fV_mat.CHm[m,:,:]
    newPIOT.loc[sec.firms,sec.institutions] = fV_mat.CGm[m,:,:]
    newPIOT.loc[sec.firms,"WASTE"] = fV_mat.WIm[m,:]
    newPIOT.loc[sec.consumers[sec.consumers!="INVESTOR"],"WASTE"] = fV_mat.WHm[m,:]
    newPIOT.loc[sec.institutions,"WASTE"] = fV_mat.WGm[m,:]
    newPIOT.loc["EXTRACTION",sec.firms] = fV_mat.Rm[m,:]
    newPIOT.loc["INVESTOR","STOCK"] = fV_mat.DeltaPSm[m]
    newPIOT.loc["Total input"] = 0
    newPIOT["Total output"] = 0
    newPIOT.loc["Total input"] = newPIOT.sum(axis = 0)
    newPIOT["Total output"] = newPIOT.sum(axis = 1)
    newPIOT.loc[sec.firms,"Activity levels (lambda)"] = cV_mat.lbd[m,:]
    return newPIOT

def writeNewFlowSAM(sec, fV, cV): #new SAM, as flows
    newFlowSAM = pd.DataFrame(columns = np.concatenate((sec.firms,sec.factors,sec.consumers,sec.institutions)), index = np.concatenate((sec.firms,sec.factors,sec.consumers,sec.institutions)))
    newFlowSAM.loc[sec.firms,sec.firms] = fV.X
    newFlowSAM.loc[sec.factors,sec.firms] = fV.L
    newFlowSAM.loc[sec.firms,sec.consumers] = fV.CH
    newFlowSAM.loc[sec.firms,sec.institutions] = fV.CG
    newFlowSAM.loc[sec.consumers[sec.consumers!="INVESTOR"],sec.factors] = fV.Omega
    newFlowSAM.loc[sec.institutions,sec.consumers] = fV.TH
    newFlowSAM.loc[sec.institutions,sec.firms] = fV.TI
    newFlowSAM.loc[sec.institutions,sec.institutions] = fV.TG
    newFlowSAM.loc[sec.consumers[sec.consumers=="INVESTOR"],sec.consumers[sec.consumers!="INVESTOR"]] = fV.S
    newFlowSAM.loc["Total Expenditures"] = 0
    newFlowSAM["Total Revenues"] = 0
    newFlowSAM.loc["Total Expenditures"] = newFlowSAM.sum(axis = 0)
    newFlowSAM["Total Revenues"] = newFlowSAM.sum(axis = 1)
    newFlowSAM.loc["Total Expenditures","Total Revenues"] = np.nan
    # we also add prices as a new colomn on the right, for firms and factors
    newFlowSAM["Prices"] = np.nan
    newFlowSAM.loc[sec.firms,"Prices"] = cV.pi
    newFlowSAM.loc[sec.factors,"Prices"] = cV.pf
    newFlowSAM.loc[sec.firms,"Activity levels"] = cV.y
    newFlowSAM.loc[sec.consumers,"Activity levels"] = cV.zh
    newFlowSAM.loc[sec.institutions,"Activity levels"] = cV.zg
    newFlowSAM = newFlowSAM.astype(float).round(3)
    return newFlowSAM

def writeNewValueSAM(sec, fV, cV): #new SAM, in monetary values
    newValueSAM = pd.DataFrame(columns = np.concatenate((sec.firms,sec.factors,sec.consumers,sec.institutions)), index = np.concatenate((sec.firms,sec.factors,sec.consumers,sec.institutions)))
    newValueSAM.loc[sec.firms,sec.firms] = fV.X
    newValueSAM.loc[sec.factors,sec.firms] = fV.L
    newValueSAM.loc[sec.firms,sec.consumers] = fV.CH
    newValueSAM.loc[sec.firms,sec.institutions] = fV.CG
    newValueSAM.loc[sec.consumers[sec.consumers!="INVESTOR"],sec.factors] = fV.Omega
    newValueSAM.loc[sec.institutions,sec.consumers] = fV.TH
    newValueSAM.loc[sec.institutions,sec.firms] = fV.TI
    newValueSAM.loc[sec.institutions,sec.institutions] = fV.TG
    newValueSAM.loc[sec.consumers[sec.consumers=="INVESTOR"],sec.consumers[sec.consumers!="INVESTOR"]] = fV.S
    #change with prices
    newValueSAM.loc[sec.firms] = newValueSAM.loc[sec.firms]*np.matrix(cV.pi).T
    newValueSAM.loc[sec.factors] = newValueSAM.loc[sec.factors]* np.matrix(cV.pf).T
    newValueSAM.loc[sec.consumers[sec.consumers!="INVESTOR"],sec.factors] = newValueSAM.loc[sec.consumers[sec.consumers!="INVESTOR"],sec.factors]*cV.pf
    newValueSAM.loc["Total Expenditures"] = 0
    newValueSAM["Total Revenues"] = 0
    newValueSAM["Prices"] = np.nan
    newValueSAM["Activity levels"] = np.nan
    newValueSAM.loc["Total Expenditures"] = newValueSAM.sum(axis = 0)
    newValueSAM["Total Revenues"] = newValueSAM.sum(axis = 1)
    newValueSAM.loc["Total Expenditures","Total Revenues"] = np.nan
    newValueSAM.loc[sec.firms,"Prices"] = cV.pi
    newValueSAM.loc[sec.factors,"Prices"] = cV.pf
    newValueSAM.loc[sec.firms,"Activity levels"] = cV.y
    newValueSAM.loc[sec.consumers,"Activity levels"] = cV.zh
    newValueSAM.loc[sec.institutions,"Activity levels"] = cV.zg
    newValueSAM = newValueSAM.astype(float).round(3)
    return newValueSAM

# to be adapted for wide SAMs (not sure to_latex() allows that), as it does not fit a standard \textwidth
def writeLatex(Scenarios,sec,filename,SAMtype,static=False,PIOT=False):
    with open('output/'+filename+'.tex','w') as tf:
        tf.write("\\documentclass{article}\n")
        tf.write("\\usepackage{booktabs}\n")
        tf.write("\\usepackage{longtable}\n")
        tf.write("\\begin{document}\n")
        tf.write("\n\n")
    for x in Scenarios:
        with open('output/'+filename+'.tex','a') as tf:
            if static==True:
                if SAMtype=="Value":
                    tf.write(writeNewValueSAM(sec, Scenarios[x]["Y0"].fV, Scenarios[x]["Y0"].cV).set_axis(np.concatenate((sec.get_abbr_sectors(),["Tot Rev"],["Prices"],["Act lvl"])), axis=1).style.format(na_rep="", precision=3).to_latex(caption=x+" - values",label="tab:"+x+"_values",))
                elif SAMtype=="Flow":
                    tf.write(writeNewFlowSAM(sec, Scenarios[x]["Y0"].fV, Scenarios[x]["Y0"].cV).set_axis(np.concatenate((sec.get_abbr_sectors(),["Tot Rev"],["Prices"],["Act lvl"])), axis=1).style.format(na_rep="", precision=3).to_latex(caption=x+" - flows",label="tab:"+x+"_flows"))
                elif SAMtype=="Both":
                    tf.write(writeNewValueSAM(sec, Scenarios[x]["Y0"].fV, Scenarios[x]["Y0"].cV).set_axis(np.concatenate((sec.get_abbr_sectors(),["Tot Rev"],["Prices"],["Act lvl"])), axis=1).style.format(na_rep="", precision=3).to_latex(caption=x+" - values",label="tab:"+x+"_values",))
                    tf.write("\n\n")
                    tf.write(writeNewFlowSAM(sec, Scenarios[x]["Y0"].fV, Scenarios[x]["Y0"].cV).set_axis(np.concatenate((sec.get_abbr_sectors(),["Tot Rev"],["Prices"],["Act lvl"])), axis=1).style.format(na_rep="", precision=3).to_latex(caption=x+" - flows",label="tab:"+x+"_flows"))
                else:
                    tf.write("Incorrect SAM type")
                    print("Incorrect SAM type")
                tf.write("\n\n")
                if PIOT==True:
                    for m in sec.materials:
                        tf.write(writeNewPIOT(sec, Scenarios[x]["Y0"].fV_mat, Scenarios[x]["Y0"].cV_mat,m).set_axis(np.concatenate((sec.get_abbr_sectors(),["WASTE"],["STOCK"],["OUT"],["Lambda"])), axis=1).style.format(na_rep="", precision=3).to_latex(caption=x+" - PIOT "+m,label="tab:"+x+"_PIOT_"+m,))
                        tf.write("\n\n")
            else:
                if SAMtype=="Value":
                    tf.write(writeNewValueSAM(sec, Scenarios[x].fV, Scenarios[x].cV).set_axis(np.concatenate((sec.get_abbr_sectors(),["Tot Rev"],["Prices"],["Act lvl"])), axis=1).style.format(na_rep="", precision=3).to_latex(caption=x+" - values",label="tab:"+x+"_values"))
                elif SAMtype=="Flow":
                    tf.write(writeNewFlowSAM(sec, Scenarios[x].fV, Scenarios[x].cV).set_axis(np.concatenate((sec.get_abbr_sectors(),["Tot Rev"],["Prices"],["Act lvl"])), axis=1).style.format(na_rep="", precision=3).to_latex(caption=x+" - flows",label="tab:"+x+"_flows"))
                elif SAMtype=="Both":
                    tf.write(writeNewValueSAM(sec, Scenarios[x].fV, Scenarios[x].cV).set_axis(np.concatenate((sec.get_abbr_sectors(),["Tot Rev"],["Prices"],["Act lvl"])), axis=1).style.format(na_rep="", precision=3).to_latex(caption=x+" - values",label="tab:"+x+"_values"))
                    tf.write("\n\n")
                    tf.write(writeNewFlowSAM(sec, Scenarios[x].fV, Scenarios[x].cV).set_axis(np.concatenate((sec.get_abbr_sectors(),["Tot Rev"],["Prices"],["Act lvl"])), axis=1).style.format(na_rep="", precision=3).to_latex(caption=x+" - flows",label="tab:"+x+"_flows"))
                else:
                    tf.write("Incorrect SAM type")
                    print("Incorrect SAM type")
                tf.write("\n\n")
                if PIOT==True:
                    for m in sec.materials:
                        tf.write(writeNewPIOT(sec, Scenarios[x].fV_mat, Scenarios[x].cV_mat,m).set_axis(np.concatenate((sec.get_abbr_sectors(),["WASTE"],["STOCK"],["OUT"],["Lambda"])), axis=1).style.format(na_rep="", precision=3).to_latex(caption=x+" - PIOT "+m,label="tab:"+x+"_PIOT_"+m,))
                        tf.write("\n\n")
    with open('output/'+filename+'.tex','a') as tf:
        tf.write("\n\n\\end{document}\n")

# two write in the same tex table two different scenarii, to be compared
# tabletype can be: SAM_value, SAM_flow, PIOT (then, precise material)
def writeLatex_compare(scenario1, scenario2,sec,filename,tabletype,material,static=True,year="Y0"):
    with open('output/'+filename+'.tex','w') as tf:
        tf.write("\\documentclass{article}\n")
        tf.write("\\usepackage{booktabs}\n")
        tf.write("\\usepackage{longtable}\n")
        tf.write("\\begin{document}\n")
        tf.write("\n\n")
    with open('output/'+filename+'.tex','a') as tf:
        if static==True:
            if tabletype=="SAM_value":
                df = pd.concat([writeNewValueSAM(sec, scenario1["Y0"].fV, scenario1["Y0"].cV),writeNewValueSAM(sec, scenario2["Y0"].fV, scenario1["Y0"].cV)])
                tf.write(df.set_axis(np.concatenate((sec.get_abbr_sectors(),["Tot Rev"],["Prices"],["Act lvl"])), axis=1).style.format(na_rep="", precision=3).to_latex(caption=scenario1["Y0"].name+" vs "+scenario2["Y0"].name+" - SAM values",label="tab:"+scenario1["Y0"].name+"_"+scenario2["Y0"].name+"-SAM_values"))
            elif tabletype=="SAM_flow":
                df = pd.concat([writeNewFlowSAM(sec, scenario1["Y0"].fV, scenario1["Y0"].cV),writeNewFlowSAM(sec, scenario2["Y0"].fV, scenario1["Y0"].cV)])
                tf.write(df.set_axis(np.concatenate((sec.get_abbr_sectors(),["Tot Rev"],["Prices"],["Act lvl"])), axis=1).style.format(na_rep="", precision=3).to_latex(caption=scenario1["Y0"].name+" vs "+scenario2["Y0"].name+" - SAM flows",label="tab:"+scenario1["Y0"].name+"_"+scenario2["Y0"].name+"-SAM_flows"))
            elif tabletype=="PIOT":
                df = pd.concat([writeNewPIOT(sec, scenario1["Y0"].fV_mat, scenario1["Y0"].cV_mat,material),writeNewPIOT(sec, scenario2["Y0"].fV_mat, scenario2["Y0"].cV_mat,material)])
                tf.write(df.set_axis(np.concatenate((sec.get_abbr_sectors(),["WASTE"],["STOCK"],["OUT"],["Lambda"])), axis=1).style.format(na_rep="", precision=3).to_latex(caption=scenario1["Y0"].name+" vs "+scenario2["Y0"].name+" - PIOT "+material,label="tab:"+scenario1["Y0"].name+"_"+scenario2["Y0"].name+"_PIOT_"+material,))
            else:
                tf.write("Incorrect table type")
                print("Incorrect table type")
            tf.write("\n\n")
        else:           # in that case, no caption with scen name yet, because not compiled by latex
            if tabletype=="SAM_value":
                df = pd.concat([writeNewValueSAM(sec, scenario1[year].fV, scenario1[year].cV),writeNewValueSAM(sec, scenario2[year].fV, scenario2[year].cV)])
                tf.write(df.set_axis(np.concatenate((sec.get_abbr_sectors(),["Tot Rev"],["Prices"],["Act lvl"])), axis=1).style.format(na_rep="", precision=3).to_latex(caption=" - SAM values",label="tab:"+scenario1[year].name+"_"+scenario2[year].name+"-SAM_values"))
            elif tabletype=="SAM_flow":
                df = pd.concat([writeNewFlowSAM(sec, scenario1[year].fV, scenario1[year].cV),writeNewFlowSAM(sec, scenario2[year].fV, scenario2[year].cV)])
                tf.write(df.set_axis(np.concatenate((sec.get_abbr_sectors(),["Tot Rev"],["Prices"],["Act lvl"])), axis=1).style.format(na_rep="", precision=3).to_latex(caption=" - SAM flows",label="tab:"+scenario1[year].name+"_"+scenario2[year].name+"-SAM_flows"))
            elif tabletype=="PIOT":
                df = pd.concat([writeNewPIOT(sec, scenario1[year].fV_mat, scenario1[year].cV_mat,material),writeNewPIOT(sec, scenario2[year].fV_mat, scenario2[year].cV_mat,material)])
                tf.write(df.set_axis(np.concatenate((sec.get_abbr_sectors(),["WASTE"],["STOCK"],["OUT"],["Lambda"])), axis=1).style.format(na_rep="", precision=3).to_latex(caption=" - PIOT "+material,label="tab:"+scenario1[year].name+"_"+scenario2[year].name+"_PIOT_"+material,))
            else:
                tf.write("Incorrect table type")
                print("Incorrect table type")
            tf.write("\n\n")
    with open('output/'+filename+'.tex','a') as tf:
        tf.write("\n\n\\end{document}\n")

def writeExcel(Scenarios,sec,filename,SAMtype,static=False,PIOT=False):
    writer = pd.ExcelWriter("output/"+filename+".xlsx", engine = 'xlsxwriter')
    for x in Scenarios:
        if static==True: #in case of a set of static runs, we plot every scenario in each tab. In that case, it take the full dictionary as argument
            if SAMtype=="Value":
                writeNewValueSAM(sec, Scenarios[x]["Y0"].fV, Scenarios[x]["Y0"].cV).to_excel(writer, sheet_name=x+"_SAM_Value")
            elif SAMtype=="Flow":
                writeNewFlowSAM(sec, Scenarios[x]["Y0"].fV, Scenarios[x]["Y0"].cV).to_excel(writer, sheet_name=x+"_SAM_Flow")
            elif SAMtype=="Both":
                writeNewValueSAM(sec, Scenarios[x]["Y0"].fV, Scenarios[x]["Y0"].cV).to_excel(writer, sheet_name=x+"_SAM_Value")
                writeNewFlowSAM(sec, Scenarios[x]["Y0"].fV, Scenarios[x]["Y0"].cV).to_excel(writer, sheet_name=x+"_SAM_Flow")
            else:
                print("Incorrect SAM type")
            if PIOT==True:
                for m in sec.materials:
                    writeNewPIOT(sec, Scenarios[x]["Y0"].fV_mat, Scenarios[x]["Y0"].cV_mat,m).to_excel(writer, sheet_name=x+"_PIOT_"+m)
        else:
            if SAMtype=="Value":
                writeNewValueSAM(sec, Scenarios[x].fV, Scenarios[x].cV).to_excel(writer, sheet_name=x+"_SAM_Value")
            elif SAMtype=="Flow":
                writeNewFlowSAM(sec, Scenarios[x].fV, Scenarios[x].cV).to_excel(writer, sheet_name=x+"_SAM_Flow")
            elif SAMtype=="Both":
                writeNewValueSAM(sec, Scenarios[x].fV, Scenarios[x].cV).to_excel(writer, sheet_name=x+"_SAM_Value")
                writeNewFlowSAM(sec, Scenarios[x].fV, Scenarios[x].cV).to_excel(writer, sheet_name=x+"_SAM_Flow")
            else:
                print("Incorrect SAM type")
            if PIOT==True:
                for m in sec.materials:
                    writeNewPIOT(sec, Scenarios[x].fV_mat, Scenarios[x].cV_mat,m).to_excel(writer, sheet_name=x+"_PIOT_"+m)
    writer.save()
    # writer.close()

# The graph export is coded to work for 5 graphs (size of the color and ls lists)
def graphDyn(eP,scenarios,var="",scens=["BAU"],title=None,yaxistitle=None,loc="lower left",endyear=10,ymax=0,ymin=0,savefile=None, format = 'pdf'):
    years = ["Y"+str(s) for s in list(range(0, eP.nbYear, 1))]
    # scen_list=["BAU","Welfare"]
    scen_list=scens
    # label_list=["'BAU'","'Welfare'"]
    label_list=["'"+x+"'" for x in scen_list]
    ls_list=[(0,(1,1)),(0,(3,1)),(0,(5,1)),(0,(6,1)),(0,(7,1))]
    color_list=["'black'","'navy'","'blue'","'green'","'limegreen'"]
    fig, figure = plt.subplots()
    for i in range(len(scen_list)):
        if scen_list[i] in scens:
            # print("%s %s %s %s %s %s %s %s %s %s %s %s" % ("figure.plot(list(years),[ScensDyn",[scen_list[i]],"[j]",".",var," for j in years], color=",color_list[i],",label=",label_list[i],",ls=",ls_list[i],")"))
            exec("%s %s %s %s %s %s %s %s %s %s %s %s" % ("figure.plot(list(years),[ScensDyn",[scen_list[i]],"[j]",".",var," for j in years], color=",color_list[i],",label=",label_list[i],",ls=",ls_list[i],")"))
    figure.grid(color='lightgray')
    figure.set_xlim([0,endyear])
    if title != None:
        plt.title(title)
    if ymax != 0:
        figure.set_ylim([ymin,ymax])
    if yaxistitle != None:
        plt.ylabel(yaxistitle)
    plt.legend(loc=loc)
    if savefile != None:
        plt.savefig("graphs/"+savefile+"."+format, format = format, transparent=True)
    plt.show()

### Calibration

In [38]:
# READ THE SAM

def baseline(SAM, fP):
        fV0 = flowVariables(fP)
        fV0.X = SAM.data.loc[SAM.get_firms(),SAM.get_firms()].values
        fV0.L = SAM.data.loc[SAM.get_factors(),SAM.get_firms()].values
        fV0.CH = SAM.data.loc[SAM.get_firms(),SAM.get_consumers()].values
        fV0.CG = SAM.data.loc[SAM.get_firms(),SAM.get_institutions()].values
        fV0.Omega = SAM.data.loc[[x for x in SAM.get_consumers() if x!="INVESTOR"],SAM.get_factors()].values
        fV0.S = SAM.data.loc["INVESTOR",[x for x in SAM.get_consumers() if x!="INVESTOR"]].values
        fV0.TH = SAM.data.loc[SAM.get_institutions(),SAM.get_consumers()].values
        fV0.TI = SAM.data.loc[SAM.get_institutions(),SAM.get_firms()].values
        fV0.TG = SAM.data.loc[SAM.get_institutions(),SAM.get_institutions()].values
        fV0.Y = sum(fV0.CH[:,h] for h in range(fP.H))+sum(fV0.CG[:,g] for g in range(fP.G))+sum(fV0.X[:,j] for j in range(fP.I))
        return fV0

def baseline_mat(PIOTs, SAM, fP):
        fV0_mat = flowVariables_mat(fP)
        for m in range(fP.M):
                # DO A PROCEDURE TO CHECK IF THE PIOTs MATCHES THE SAM
                fV0_mat.Xm[m,:,:] = PIOTs[m].data.loc[SAM.get_firms(),SAM.get_firms()].values
                fV0_mat.CHm[m,:,:] = PIOTs[m].data.loc[SAM.get_firms(),SAM.get_consumers()].values
                fV0_mat.CGm[m,:,:] = PIOTs[m].data.loc[SAM.get_firms(),SAM.get_institutions()].values
                fV0_mat.Ym[m,:] = sum(fV0_mat.CHm[m,:,h] for h in range(fP.H))+sum(fV0_mat.CGm[m,:,g] for g in range(fP.G))+sum(fV0_mat.Xm[m,:,j] for j in range(fP.I))
                fV0_mat.WIm[m,:] = PIOTs[m].data.loc[SAM.get_firms(),"WASTE"].values
                fV0_mat.WHm[m,:] = PIOTs[m].data.loc[[x for x in SAM.get_consumers() if x!="INVESTOR"],"WASTE"].values
                fV0_mat.WGm[m,:] = PIOTs[m].data.loc[SAM.get_institutions(),"WASTE"].values
                fV0_mat.Rm[m,:] = PIOTs[m].data.loc["EXTRACTION",SAM.get_firms()].values
                fV0_mat.DeltaPSm[m] = PIOTs[m].data.loc["INVESTOR","STOCK"]
        return fV0_mat
        

In [39]:
# CALIBRATION 

# no prices as they are all equal to one in the baseline

def funcInitPolicy(fP,SAM_xls,SAM_sheet,sec,fV0):
    policytab = pd.read_excel(SAM_xls,sheet_name=SAM_sheet,index_col=1).drop(['Unnamed: 0'], axis=1)
    materials = pd.read_excel(SAM_xls,sheet_name='MATERIALS',usecols='A')
    pV0 = policyVariables(fP)
    policiesX = policytab.loc[sec.firms,'SECTORIAL'].values
    policiesH = policytab.loc[sec.consumers,'SECTORIAL'].values
    policiesR = policytab.loc[sec.firms,['RESOURCE_'+m for m in materials["Material"].values]].values
    policiesWI = policytab.loc[sec.firms,['WASTE_'+m for m in materials["Material"].values]].values
    policiesWH = policytab.loc[sec.consumers,['WASTE_'+m for m in materials["Material"].values]].values
    policiesG = policytab.loc[sec.institutions,'SECTORIAL'].values
    policiesWG = policytab.loc[sec.institutions,['WASTE_'+m for m in materials["Material"].values]].values
    for i in range(fP.I):
        pV0.tauX[i] = policiesX[i]
        pV0.tauR[:,i] = policiesR[i,:]
        pV0.tauWI[:,i] = policiesWI[i,:]
    for h in range(fP.H):
        if h==fP.investor:
            continue
        else:
            pV0.tauH[h] = policiesH[h]
            pV0.tauWH[:,h] = policiesWH[h,:]
    for g in range(fP.G):
        pV0.tauG[g] = policiesG[g]
        pV0.tauWG[:,g] = policiesWG[g,:]
    return pV0

def funcInitControlV(fP, fV0, fV0_mat):
    cV0 = controlVariables(fP)
    cV0_mat = controlVariables_mat(fP)
    iV0 = intensityVariables(fP)
    # econ control V
    cV0.y = np.full(fP.I, 1.)
    cV0.zh = np.full(fP.H, 1.)
    cV0.zg = np.full(fP.G, 1.)
    cV0.pi = np.full(fP.I, 1.)
    cV0.pf = np.full(fP.F, 1.)
    # mat control V
    cV0_mat.lbd = np.full((fP.M,fP.I), 1.)
    # intensity V
    for m in range(fP.M):
        for i in range(fP.I):
            for j in range(fP.I):
                iV0.thetaX[m,i,j] = fV0.X[i,j] and fV0_mat.Xm[m,i,j]/fV0.X[i,j] or 0
        for i in range(fP.I):
            for h in range(fP.H):
                iV0.thetaCH[m,i,h] = fV0.CH[i,h] and fV0_mat.CHm[m,i,h]/fV0.CH[i,h] or 0
        for i in range(fP.I):
            for g in range(fP.G):
                iV0.thetaCG[m,i,g] = fV0.CG[i,g] and fV0_mat.CGm[m,i,g]/fV0.CG[i,g] or 0
        # for i in range(fP.I):
        #     iV0.thetaY[m,i] = fV0.Y[i] and fV0_mat.Ym[m,i]/fV0.Y[i] or 0
    return [cV0, cV0_mat, iV0]

def funcInitCapital(fP, fV0_mat, fV0, cP):
    sV0 = stockVariables_mat(fP)
    sV0.Km = (sum(fV0_mat.CHm[:,i,fP.investor] for i in range(fP.I))/(cP.Ah[fP.investor]*np.prod(np.power(fV0.CH[:,fP.investor],cP.gamma[:,fP.investor]))))*sum( fV0.Omega[:,fP.capital])
    return sV0

def funcCalibrate(fP,iV0,fV0,fV0_mat,pV0,display=False):
    cP = calibParameters(fP)
    for m in range(fP.M):
        for i in range(fP.I):
            cP.epsilon[m,i] = (sum(fV0_mat.Xm[m,:,i])+fV0_mat.Rm[m,i]) and fV0_mat.WIm[m,i]/(sum(fV0_mat.Xm[m,:,i])+fV0_mat.Rm[m,i]) or 0
    for m in range(fP.M):
        for i in range(fP.I):
            cP.rho[m,i] = (fV0.Y[i]) and fV0_mat.Rm[m,i]/(fV0.Y[i]) or 0
    for i in range(fP.I):
        cP.alpha[:,i] = (((1+pV0.tauX)+sum(cP.epsilon[m,i]*iV0.thetaX[m,:,i]*pV0.tauWI[m,i] for m in range(fP.M)))*fV0.X[:,i]/fV0.Y[i])/(1-sum(cP.rho[:,i]*pV0.tauR[:,i])-sum(cP.epsilon[:,i]*cP.rho[:,i]*pV0.tauWI[:,i]))
    for i in range(fP.I):
        cP.beta[:,i] = fV0.L[:,i]/(fV0.Y[i]*(1-sum(cP.rho[:,i]*pV0.tauR[:,i])-sum(cP.epsilon[:,i]*cP.rho[:,i]*pV0.tauWI[:,i])))
    for h in range(fP.H):
        if h==fP.investor:
            cP.sigma[h] = 0
        else:
            cP.sigma[h] = fV0.S[h]/sum(fV0.Omega[h,:])
    for h in range(fP.H):
        if h==fP.investor:
            cP.gamma[:,h] = (1+pV0.tauX)*fV0.CH[:,h]/(sum((1+pV0.tauX)*fV0.CH[:,h]))
        else:
            cP.gamma[:,h] = ((1+pV0.tauH[h]+pV0.tauX)+sum(iV0.thetaCH[m,:,h]*pV0.tauWH[m,h] for m in range(fP.M)))*fV0.CH[:,h]/(sum(((1+pV0.tauH[h]+pV0.tauX)+sum(iV0.thetaCH[m,:,h]*pV0.tauWH[m,h] for m in range(fP.M)))*fV0.CH[:,h]))
    for g in range(fP.G):
        cP.eta[:,g] = ((1+pV0.tauG[g]+pV0.tauX)+sum(iV0.thetaCG[m,:,g]*pV0.tauWG[m,g] for m in range(fP.M)))*fV0.CG[:,g]/sum(((1+pV0.tauG[g]+pV0.tauX)+sum(iV0.thetaCG[m,:,g]*pV0.tauWG[m,g] for m in range(fP.M)))*fV0.CG[:,g])
    for i in range(fP.I):
        cP.Ai[i] = sum(-x*np.log(x) for x in cP.alpha[:,i] if x>0) + sum(cP.alpha[:,i]*np.log((1+pV0.tauX)+sum(cP.epsilon[m,i]*iV0.thetaX[m,:,i]*pV0.tauWI[m,i] for m in range(fP.M)))) + sum(-x*np.log(x) for x in cP.beta[:,i] if x> 0) - np.log(1-sum(cP.rho[:,i]*pV0.tauR[:,i])-sum(cP.epsilon[:,i]*cP.rho[:,i]*pV0.tauWI[:,i]))
    cP.Ai = np.exp(cP.Ai)
    for h in range(fP.H):
        if h==fP.investor:
            cP.Ah[h] = sum(-x*np.log(x) for x in cP.gamma[:,h] if x>0) + sum(cP.gamma[:,h]*np.log(1+pV0.tauX))
        else:
            cP.Ah[h] = sum(x*(-np.log(x)) for x in cP.gamma[:,h] if x>0) + sum(cP.gamma[:,h]*np.log((1+pV0.tauX+pV0.tauH[h])+sum(iV0.thetaCH[m,:,h]*pV0.tauWH[m,h] for m in range(fP.M))))
    cP.Ah = np.exp(cP.Ah)
    for g in range(fP.G):
        cP.Ag[g] = sum(-x*np.log(x) for x in cP.eta[:,g] if x>0) + sum(cP.eta[:,g]*np.log((1+pV0.tauX+pV0.tauG[g])+sum(iV0.thetaCG[m,:,g]*pV0.tauWG[m,g] for m in range(fP.M))))
    cP.Ag = np.exp(cP.Ag)
    for h in range(fP.H):
        if h==fP.investor:
            continue
        else:
            for f in range(fP.F):
                if f==fP.capital:
                    continue
                else:
                    cP.Omega[h,f] = fV0.Omega[h,f]   # inelastic labor supply
    for i in range(fP.I):
        for m in range(fP.M):
            if fV0_mat.Ym[m,i]==0.:
                cP.bl[m,i] = True
    if display:
        print(  "alpha: ",cP.alpha)
        print(  "beta:  ",cP.beta)
        print(  "gamma: ",cP.gamma)
        print(  "eta:   ",cP.eta)
        print(  "Ai:  ",cP.Ai)
        print(  "Ah:  ",cP.Ah)
        print(  "Ag:  ",cP.Ag)
        print(  "Omega:  ",cP.Omega)
        print(  "sigma:  ",cP.sigma)
        print(  "epsilon:  ",cP.epsilon)
        print(  "rho:  ",cP.rho)
        print(  "Array Lambda: ", cP.bl)
    return cP



### Variables mappings

In [40]:
# MAPPINGS

class mappingFOCI:
    def __init__(self, fP):
        self.psiI = np.full(fP.I, np.nan, float)
        self.x = np.full((fP.I,fP.I), np.nan, float)
        self.l = np.full((fP.F,fP.I), np.nan, float)

class mappingFOCH:
    def __init__(self, fP):
        self.psiH = np.full(fP.H, np.nan, float)
        self.cH = np.full((fP.I,fP.H), np.nan, float)
   
class mappingFOCG:
    def __init__(self, fP):
        self.psiG = np.full(fP.G, np.nan, float)
        self.cG = np.full((fP.I,fP.G), np.nan, float)

def FOC_I(fP, cV, pV, iV, cP):
    mapI = mappingFOCI(fP)
    psiItilde = np.full(fP.I, np.nan, float)
    for i in range(fP.I):
        psiItilde[i] = sum(cP.alpha[:,i]*np.log((1+pV.tauX)*cV.pi+sum(cP.epsilon[m,i]*iV.thetaX[m,:,i]*pV.tauWI[m,i] for m in range(fP.M)))) + sum(-x*np.log(x) for x in cP.alpha[:,i] if x>0)+ sum(cP.beta[:,i]*(np.log(cV.pf))) + sum(-x*np.log(x) for x in cP.beta[:,i] if x>0)-np.log(cP.Ai[i])
    psiItilde = np.exp(psiItilde)
    mapI.psiI = psiItilde + sum(cP.rho[m,:]*pV.tauR[m,:] for m in range(fP.M))+sum(cP.epsilon[m,:]*cP.rho[m,:]*pV.tauWI[m,:] for m in range(fP.M)) # return Psi as this will be equal to pi the zero profit condition
    for i in range(fP.I):
        for j in range(fP.I):
            mapI.x[j,i] = cP.alpha[j,i]*psiItilde[i]/((1+pV.tauX[j])*cV.pi[j]+sum(cP.epsilon[m,i]*iV.thetaX[m,j,i]*pV.tauWI[m,i] for m in range(fP.M)))
        mapI.l[:,i] = cP.beta[:,i]*psiItilde[i]/cV.pf
    return mapI

def FOC_H(fP, cV, pV, iV, cP):
    mapH = mappingFOCH(fP)
    for h in range(fP.H):
        if h==fP.investor:
            mapH.psiH[h] = sum(cP.gamma[:,h]*np.log((1+pV.tauX)*cV.pi))+sum(-x*np.log(x) for x in cP.gamma[:,h] if x>0)-np.log(cP.Ah[h])
        else:
            mapH.psiH[h] = sum(cP.gamma[:,h]*np.log((1+pV.tauX+pV.tauH[h])*cV.pi+sum(iV.thetaCH[m,:,h]*pV.tauWH[m,h] for m in range(fP.M))))+sum(-x*np.log(x) for x in cP.gamma[:,h] if x>0)-np.log(cP.Ah[h])
    mapH.psiH = np.exp(mapH.psiH)
    for h in range(fP.H):
        if h==fP.investor:
            mapH.cH[:,h] = cP.gamma[:,h]*mapH.psiH[h]/((1+pV.tauX)*cV.pi)
        else:
            mapH.cH[:,h] = cP.gamma[:,h]*mapH.psiH[h]/((1+pV.tauX+pV.tauH[h])*cV.pi+sum(iV.thetaCH[m,:,h]*pV.tauWH[m,h] for m in range(fP.M)))
    return mapH

def FOC_G(fP, cV, pV, iV, cP):
    mapG = mappingFOCG(fP)
    for g in range(fP.G):
        mapG.psiG[g] = sum(cP.eta[:,g]*np.log((1+pV.tauX+pV.tauG[g])*cV.pi+sum(iV.thetaCG[m,:,g]*pV.tauWG[m,g] for m in range(fP.M))))+sum(-x*np.log(x) for x in cP.eta[:,g] if x>0)-np.log(cP.Ag[g])
    mapG.psiG = np.exp(mapG.psiG)
    for g in range(fP.G):
        mapG.cG[:,g] = cP.eta[:,g]*mapG.psiG[g]/((1+pV.tauX+pV.tauG[g])*cV.pi+sum(iV.thetaCG[m,:,g]*pV.tauWG[m,g] for m in range(fP.M)))
    return mapG

class mappingPROD:
    def __init__(self, fP):
        self.psiI = np.full(fP.I, np.nan, float)
        self.X = np.full((fP.I,fP.I), np.nan, float)
        self.L = np.full((fP.F,fP.I), np.nan, float)
        self.Y = np.full(fP.I, np.nan, float)

class mappingCONSH:
    def __init__(self, fP):
        self.CH = np.full((fP.I,fP.H), np.nan, float)

class mappingCONSG:
    def __init__(self, fP):
        self.CG = np.full((fP.I,fP.G), np.nan, float)

def PROD(fP, cV, pV, iV, cP, fV0):
    mapPROD = mappingPROD(fP)
    mapI = FOC_I(fP, cV, pV, iV, cP)
    mapPROD.psiI = mapI.psiI
    # here a for loop because row and column multiplication (lazy solution)
    for i in range(fP.I):
        mapPROD.Y[i] = cV.y[i]*fV0.Y[i]
    for i in range(fP.I):
        mapPROD.X[:,i] = cV.y[i]*fV0.Y[i]*mapI.x[:,i]
    for f in range(fP.F):
        for i in range(fP.I):
            mapPROD.L[f,i] = cV.y[i]*fV0.Y[i]*mapI.l[f,i]
    return mapPROD

def CONS_H(fP, cV, pV, iV, cP, fV0):
    mapCONSH = mappingCONSH(fP)
    mapH = FOC_H(fP, cV, pV, iV, cP)
    for h in range(fP.H):
        for i in range(fP.I):
            mapCONSH.CH[i,h] = cV.zh[h]*mapH.cH[i,h]*(cP.Ah[h]*np.prod(np.power(fV0.CH[:,h],cP.gamma[:,h])))
    return mapCONSH

def CONS_G(fP, cV, pV, iV, cP, fV0):
    mapCONSG = mappingCONSG(fP)
    mapG = FOC_G(fP, cV, pV, iV, cP)
    for g in range(fP.G):
        for i in range(fP.I):
            mapCONSG.CG[i,g] = cV.zg[g]*mapG.cG[i,g]*(cP.Ag[g]*np.prod(np.power(fV0.CG[:,g],cP.eta[:,g])))
    return mapCONSG

def funcSimulate(fP, cP, fV0, pV, iV, fVnew, cV): #argument fVnew corresponds to the new imposed value of capital revenue
    # careful : now producing fV involves having intensity variables, produced by MAT(), but BEFORE producing fV_mat (that needs fV and MAT)
    fV = flowVariables(fP)
    fV.X = PROD(fP, cV, pV, iV, cP, fV0).X
    fV.L = PROD(fP, cV, pV, iV, cP, fV0).L
    fV.CH = CONS_H(fP, cV, pV, iV, cP, fV0).CH
    fV.CG = CONS_G(fP, cV, pV, iV, cP, fV0).CG
    fV.Y = PROD(fP, cV, pV, iV, cP, fV0).Y
    for h in range(fP.H):   #We adjust to previous capital
        if h==fP.investor:
            continue
        else:
            for f in range(fP.F):
                if f==fP.capital:
                    fV.Omega[h,f] = fVnew.Omega[h,f]
                else:
                    fV.Omega[h,f] = cP.Omega[h,f]
    for h in range(fP.H):
        if h==fP.investor:
            continue
        else:
            fV.S[h] = cP.sigma[h]*sum(cV.pf*fV.Omega[h,:])
    for g in range(fP.G):
        for h in range(fP.H):
            if h==fP.investor:
                fV.TH[g,h] = sum((pV.tauX[i])*cV.pi[i]*fV.CH[i,h] for i in range(fP.I))
            else:
                fV.TH[g,h] = sum(((pV.tauX[i]+pV.tauH[h])*cV.pi[i]+sum(pV.tauWH[:,h]*iV.thetaCH[:,i,h]))*fV.CH[i,h] for i in range(fP.I))
        for j in range(fP.I):
            fV.TI[g,j] = sum((pV.tauX[i]*cV.pi[i]+sum(pV.tauWI[:,j]*cP.epsilon[:,j]*iV.thetaX[:,i,j]))*fV.X[i,j] for i in range(fP.I))+sum((pV.tauR[:,j]+pV.tauWI[:,j]*cP.epsilon[:,j])*cP.rho[:,j])*fV.Y[j]
        for g2 in range(fP.G):
            fV.TG[g,g2] = sum(((pV.tauX[i]+pV.tauG[g2])*cV.pi[i]+sum(pV.tauWG[:,g2]*iV.thetaCG[:,i,g2]))*fV.CG[i,g] for i in range(fP.I))
    return fV




In [41]:
class mapping_MAT:
    def __init__(self, fP):
        self.thetaX = np.full((fP.M,fP.I,fP.I), np.nan, float)
        self.thetaCH = np.full((fP.M,fP.I,fP.H), np.nan, float)
        self.thetaCG = np.full((fP.M,fP.I,fP.G), np.nan, float)
        # self.thetaY = np.full((fP.M,fP.I), np.nan, float)

# this mapping is important to consider material taxes: must be generated before each use of funcSimulate() !!
def MAT(fP, cV_mat, iV0):
    iV = intensityVariables(fP)
    for m in range(fP.M):     # ugly loops for now because I want to be sure it does what i want it to do !
        for j in range(fP.I):
            for i in range(fP.I):
                iV.thetaX[m,i,j] = cV_mat.lbd[m,i]*iV0.thetaX[m,i,j]
        for h in range(fP.H):
            for i in range(fP.I):
                iV.thetaCH[m,i,h] = cV_mat.lbd[m,i]*iV0.thetaCH[m,i,h]
        for g in range(fP.G):
            for i in range(fP.I):
                iV.thetaCG[m,i,g] = cV_mat.lbd[m,i]*iV0.thetaCG[m,i,g]
        # for i in range(fP.I):
        #     iV.thetaY[m,i] = cV_mat.lbd[m,i]*iV0.thetaY[m,i]
    return iV

def funcSimulate_mat(fP, cP, iV0, fV0_mat, cV_mat, fV):
    fV_mat = flowVariables_mat(fP)
    iV = MAT(fP, cV_mat, iV0)
    fV_mat.Xm = iV.thetaX*fV.X
    fV_mat.CHm = iV.thetaCH*fV.CH
    fV_mat.CGm = iV.thetaCG*fV.CG
    # fV_mat.Ym = iV.thetaY*fV.Y
    fV_mat.Ym = sum(fV_mat.Xm[:,:,j] for j in range(fP.I)) + sum(fV_mat.CHm[:,:,h] for h in range(fP.H)) + sum(fV_mat.CGm[:,:,g] for g in range(fP.G))
    for m in range(fP.M):    #alternative version
        fV_mat.Rm[m,:] = cP.rho[m,:]*fV.Y
    fV_mat.WIm = cP.epsilon*(fV_mat.Rm+sum(fV_mat.Xm[:,j,:] for j in range(fP.I)))
    for h in range(fP.H):
        if h==fP.investor:
            continue
        else:
            fV_mat.WHm[:,h] = sum(fV_mat.CHm[:,i,h] for i in range(fP.I))
    fV_mat.WGm = sum(fV_mat.CGm[:,i,:] for i in range(fP.I))
    fV_mat.DeltaPSm = sum(fV_mat.CHm[:,i,fP.investor] for i in range(fP.I))
    return [iV,fV_mat]

### Balances and budgets

In [42]:
# BALANCES

def COMBAL_I(fP, fV, display = False):
    DI = np.full(fP.I, np.nan, float)
    for i in range(fP.I):
        DI[i] = np.log(sum(fV.X[i,:])+sum(fV.CH[i,:])+sum(fV.CG[i,:]))-np.log(fV.Y[i])
    if display == True:
        print("Goods balance:   ", DI)
    return DI

def COMBAL_Ii(fP, fV, display = False): #with walras law, one less good to equilibrate
    DI = np.full(fP.I-1, np.nan, float)
    for i in range(fP.I-1):
        DI[i] = np.log(sum(fV.X[i,:])+sum(fV.CH[i,:])+sum(fV.CG[i,:]))-np.log(fV.Y[i])
    if display == True:
        print("Goods balance -i:   ", DI)
    return DI

def COMBAL_F(fP, fV, cP, display = False):
    DF = np.full(fP.F, np.nan, float)
    for f in range(fP.F):
        DF[f] = np.log(sum(fV.L[f,:]))-np.log(sum(fV.Omega[:,f]))
        # DF[f] = (sum(fV.L[f,:]))-(sum(fV.Omega[:,f]))
    if display == True:
        print("Factors balance:    ", DF)
    return DF

def ZERO_PROFIT(fP, cV, pV, iV, cP, display = False):
    EI = np.full(fP.I, np.nan, float)
    for i in range(fP.I):
        EI[i] = np.log(FOC_I(fP, cV, pV, iV, cP).psiI[i])-np.log(cV.pi[i])
    if display == True:
        print("Zero profit condition:     ", EI)
    return EI

def BUD_H(fP, fV, cV, pV, cP, display = False):
    BH = np.full(fP.H, np.nan, float)
    for h in range(fP.H):
        if h==fP.investor:
            BH[h] = np.log(sum(fV.S))-np.log(sum(cV.pi*fV.CH[:,h])+sum(fV.TH[:,h]))
        else:
            BH[h] = np.log((1-cP.sigma[h])*sum(cV.pf*fV.Omega[h,:]))-np.log(sum(cV.pi*fV.CH[:,h])+sum(fV.TH[:,h]))
    if display == True:
        print("Households budgets:    ", BH)
    return BH

def BUD_G(fP, fV, cV, pV, iV, cP, display = False):
    BG = np.full(fP.G, np.nan, float)
    for g in range(fP.G):
        BG[g] = np.log(sum(fV.TH[g,:])+sum(fV.TI[g,:])+sum(fV.TG[g,:])) - np.log(sum(cV.pi*fV.CG[:,g])+sum(fV.TG[:,g]))
    if display == True:
        print("Government balance:    ", BG)
    return BG

def INF(fP, cV, pV, iV, cP, display = False): #CAREFUL THE INFLATIONARY TARGET IS FOR ONE HH ONLY !!!
    EH = np.full(1, np.nan, float)
    EH[0] = np.log(FOC_H(fP, cV, pV, iV, cP).psiH[0])
    if display == True:
        print("Inflation (hh):    ", EH)
    return EH

def INF2(fP, cV, pV, iV, cP, display = False):
    EH = np.full(1, np.nan, float)
    EH[0] = np.log(FOC_H(fP, cV, pV, iV, cP).psiH[0]) + sum(np.log(FOC_G(fP, cV, pV, iV, cP).psiG))
    if display == True:
        print("Inflation (hh+gov):    ", EH)
    return EH

def FLOWS(fP, fV_mat, display = False):
    D_IO = np.full((fP.M,fP.I), np.nan, float)
    IN = sum(fV_mat.Xm[:,j,:] for j in range(fP.I))+fV_mat.Rm
    OUT = sum(fV_mat.Xm[:,:,j] for j in range(fP.I)) + sum(fV_mat.CHm[:,:,h] for h in range(fP.H)) + sum(fV_mat.CGm[:,:,g] for g in range(fP.G)) + fV_mat.WIm
    D_IO = np.log(IN,where=(IN!=0)) - np.log(OUT,where=(OUT!=0))
    # D_IO = sum(fV_mat.Xm[:,j,:] for j in range(fP.I))+fV_mat.Rm - fV_mat.WIm - sum(fV_mat.Xm[:,:,j] for j in range(fP.I)) - sum(fV_mat.CHm[:,:,h] for h in range(fP.H)) - sum(fV_mat.CGm[:,:,g] for g in range(fP.G))
    if display == True:
        print("Material balance in/out:    ", D_IO)
    return D_IO

def EQUILIBRIUM(fP, cP, cV, cV_mat, pV, fV0, iV0, fV0_mat, fVnew, inflation = 1, display = False):
    # EQUIL = np.full((2+fP.M)*fP.I+fP.F+fP.H+fP.G+1, np.nan, float)
    EQUIL = np.full(2*fP.I+fP.F+fP.H+fP.G+1, np.nan, float)
    iV = MAT(fP, cV_mat, iV0)
    fV = funcSimulate(fP, cP, fV0, pV, iV, fVnew, cV)
    [iV,fV_mat] = funcSimulate_mat(fP, cP, iV0, fV0_mat, cV_mat, fV)
    EQUIL[0:fP.I] = COMBAL_I(fP, fV, display = display) #we also give the excess demand D_i that should be zero (Walras law)
    EQUIL[fP.I:fP.I+fP.F] = COMBAL_F(fP, fV, cP, display = display)
    EQUIL[fP.I+fP.F:fP.I+fP.F+fP.H] = BUD_H(fP, fV, cV, pV, cP, display = display)
    EQUIL[fP.I+fP.F+fP.H:fP.I+fP.F+fP.H+fP.G] = BUD_G(fP, fV, cV, pV, iV, cP, display = display)
    if inflation == 1:
        EQUIL[fP.I+fP.F+fP.H+fP.G:fP.I+fP.F+fP.H+fP.G+1] = INF(fP, cV, pV, iV, cP, display = display)
    elif inflation == 2:
        EQUIL[fP.I+fP.F+fP.H+fP.G:fP.I+fP.F+fP.H+fP.G+1] = INF2(fP, cV, pV, iV, cP, display = display)
    EQUIL[fP.I+fP.F+fP.H+fP.G+1:2*fP.I+fP.F+fP.H+fP.G+1] = ZERO_PROFIT(fP, cV, pV, iV, cP, display = display)
    EQUIL = np.concatenate((EQUIL,FLOWS(fP, fV_mat, display = display).flatten()))
    # for m in range(fP.M):
    #     EQUIL[(2+m)*fP.I+fP.F+fP.H+fP.G+1:(3+m)*fP.I+fP.F+fP.H+fP.G+1] = FLOWS(fP, fV_mat)[m,:]
    return EQUIL

### Dynamics of capital

In [43]:
# dynamics

def computeCapital(fP, fVprevious, cP, eP):
    # uses fVprevious as stored flow variables from the previous period
    # eP as exogenous parameters
    fVnew = flowVariables(fP)
    for h in range(fP.H):
        if h==fP.investor:
            continue
        else:
            fVnew.Omega[h,fP.capital] = (1-eP.delta)*fVprevious.Omega[h,fP.capital] + fVprevious.S[h]*cP.Ah[fP.investor]*np.prod(np.power(fVprevious.CH[:,fP.investor],cP.gamma[:,fP.investor]))/sum(fVprevious.S)
    return fVnew

# calculates what is depreciated in the capital ; then it updates the amount of material embedded in capital.
def deltaCapital(fP, eP, sV_mat, fV_mat, fV):
    sV_mat_new = stockVariables_mat(fP)
    fV_mat_previous = copy.deepcopy(fV_mat) # we must add the depreciation that has not been computed by the simulate function
    # fV_mat_new.DeltaPSm = sum(fV_mat.CHm[:,fP.investor])
    fV_mat_previous.DeltaMSm = eP.deltam*sV_mat.Km
    for i in range(fP.I):
        fV_mat_previous.WKm[:,i] = fV_mat_previous.DeltaMSm*fV.L[fP.capital,i]/(sum(fV.L[fP.capital,j] for j in range(fP.I)))
    sV_mat_new.Km = sV_mat.Km + fV_mat_previous.DeltaPSm - fV_mat_previous.DeltaMSm
    return [sV_mat_new,fV_mat_previous]

## Equilibriums

### Exogenous policies

Homotopy

In [44]:
def equilibrium(fP, cV, cV_mat, pV, cP, fV0, iV0, fV0_mat, fVnew):
    iV = MAT(fP, cV_mat, iV0)
    fV = funcSimulate(fP, cP, fV0, pV, iV, fVnew, cV)
    [iV,fV_mat] = funcSimulate_mat(fP, cP, iV0, fV0_mat, cV_mat, fV)
    equilibrium = np.concatenate((COMBAL_Ii(fP, fV), COMBAL_F(fP, fV, cP), BUD_H(fP, fV, cV, pV, cP), BUD_G(fP, fV, cV, pV, iV, cP), INF(fP, cV, pV, iV, cP), ZERO_PROFIT(fP, cV, pV, iV, cP)))
    for m in range(fP.M):
        for i in range(fP.I):
            if cP.bl[m,i]==False:
                equilibrium = np.concatenate((equilibrium,[FLOWS(fP, fV_mat)[m,i]]))
    return equilibrium

def equilibriumX(x, fP, cP, fV0, iV0, fV0_mat, fVnew, pVBAU, pVtarget):
    cV = controlVariables(fP)
    mu = x[0] # control variable but outside cV
    pV = policyVariables(fP)
    pV.tauX = mu*pVBAU.tauX + (1-mu)*pVtarget.tauX
    pV.tauH = mu*pVBAU.tauH + (1-mu)*pVtarget.tauH
    pV.tauG = mu*pVBAU.tauG + (1-mu)*pVtarget.tauG
    pV.tauR = mu*pVBAU.tauR + (1-mu)*pVtarget.tauR
    pV.tauWI = mu*pVBAU.tauWI + (1-mu)*pVtarget.tauWI
    pV.tauWH = mu*pVBAU.tauWH + (1-mu)*pVtarget.tauWH
    pV.tauWG = mu*pVBAU.tauWG + (1-mu)*pVtarget.tauWG
    cV_mat = controlVariables_mat(fP)
    cV.y[0:fP.I] = x[1:fP.I+1]
    cV.zh[0:fP.H] = x[fP.I+1:fP.I+fP.H+1]
    cV.zg[0:fP.G] = x[fP.I+fP.H+1:fP.I+fP.H+fP.G+1]
    cV.pi[0:fP.I] = x[fP.I+fP.H+fP.G+1:2*fP.I+fP.H+fP.G+1]
    cV.pf[0:fP.F] = x[2*fP.I+fP.H+fP.G+1:2*fP.I+fP.H+fP.G+fP.F+1]
    index = 0
    for m in range(fP.M):
        for i in range(fP.I):
            if cP.bl[m,i]==False:
                cV_mat.lbd[m,i] = x[2*fP.I+fP.H+fP.G+fP.F+1+index]
                index += 1
            else:
                cV_mat.lbd[m,i] = 1.
    return equilibrium(fP, cV, cV_mat, pV, cP, fV0, iV0, fV0_mat, fVnew)

def funcObjectiveX(x): # it is better to call a function than directly the variable x[0]
    return x[0]

def funcEquilibrium(fP, cV0, cV0_mat, cP, fV0, iV0, fV0_mat, fVnew, pVBAU, pVtarget, display=True):
    x = np.full(fP.I*(2+fP.M)+fP.H+fP.G+fP.F+1, 1.)
    x[0] = 1. #(mu starting point is BAU or previous year)
    x[1:fP.I+1] = cV0.y[0:fP.I]
    x[fP.I+1:fP.I+fP.H+1] = cV0.zh[0:fP.H]
    x[fP.I+fP.H+1:fP.I+fP.H+fP.G+1] = cV0.zg[0:fP.G]
    x[fP.I+fP.H+fP.G+1:2*fP.I+fP.H+fP.G+1] = cV0.pi[0:fP.I]
    x[2*fP.I+fP.H+fP.G+1:2*fP.I+fP.H+fP.G+fP.F+1] = cV0.pf[0:fP.F]
    for m in range(fP.M):
        for i in range(fP.I):
            if cP.bl[m,i]==False:
                x = np.concatenate((x,[cV0_mat.lbd[m,i]]))
    x_bound = [(0.00000001*x[i],2.*x[i]) for i in range(len(x)-1)]
    x_bound = np.concatenate(([(0.,1.)],x_bound))
    # test if equilibriumX returns zero for mu=1 (if not, it means initial exogenous data wrong)
    # to be adjusted: does not work when non stationary
    # testEquil = equilibriumX(x, fP, cP, fV0, iV0, fV0_mat, fVnew, pVBAU, pVtarget)
    # testEquil = sum(testEquil*testEquil)
    # if testEquil>0.000001:
    #     print('Norm of the equil is', testEquil)
    #     raise TypeError('Issue with the data: initial equilibrium is not found')
    Optimised = minimize(funcObjectiveX, x, args=(),method='SLSQP', bounds = tuple(x_bound), options={'disp': display}, constraints=[{"type":"eq", "fun":equilibriumX,"args":(fP, cP, fV0, iV0, fV0_mat, fVnew, pVBAU, pVtarget)}])
    cV = controlVariables(fP)
    cV_mat = controlVariables_mat(fP)
    mu = Optimised.x[0]
    cV.y[0:fP.I] = Optimised.x[1:fP.I+1]
    cV.zh[0:fP.H] = Optimised.x[fP.I+1:fP.I+fP.H+1]
    cV.zg[0:fP.G] = Optimised.x[fP.I+fP.H+1:fP.I+fP.H+fP.G+1]
    cV.pi[0:fP.I] = Optimised.x[fP.I+fP.H+fP.G+1:2*fP.I+fP.H+fP.G+1]
    cV.pf[0:fP.F] = Optimised.x[2*fP.I+fP.H+fP.G+1:2*fP.I+fP.H+fP.G+fP.F+1]
    index = 0
    for m in range(fP.M):
        for i in range(fP.I):
            if  cP.bl[m,i]==False:
                cV_mat.lbd[m,i] = Optimised.x[2*fP.I+fP.H+fP.G+fP.F+1+index]
                index += 1
            else:
                cV_mat.lbd[m,i] = 1.
    return [mu,cV,cV_mat]

Maintaining government consumption

In [45]:
# HOMOTOPY WITH GOV TARGET

def equilibriumGOV(fP, cV, cV_mat, pV, cP, fV0, iV0, fV0_mat, fVnew):
    iV = MAT(fP, cV_mat, iV0)
    fV = funcSimulate(fP, cP, fV0, pV, iV, fVnew, cV)
    [iV,fV_mat] = funcSimulate_mat(fP, cP, iV0, fV0_mat, cV_mat, fV)
    equilibrium = np.concatenate((COMBAL_Ii(fP, fV), COMBAL_F(fP, fV, cP), BUD_H(fP, fV, cV, pV, cP), BUD_G(fP, fV, cV, pV, iV, cP), INF(fP, cV, pV, iV, cP), ZERO_PROFIT(fP, cV, pV, iV, cP)))
    GOVCONSO = np.full(fP.G, np.nan, float)
    # share target
    GOVCONSO = np.log(sum(cV.pi[i]*fV.CG[i,:] for i in range(fP.I))) - np.log(sum(fV0.CG[i,:] for i in range(fP.I))/(sum(sum(fV0.CH[i,:] for i in range(fP.I)))+sum(sum(fV.CG[i,:] for i in range(fP.I)))+sum(fV0.TH)+sum(fV0.TG))) - np.log(sum(sum(cV.pi[i]*fV.CH[i,:] for i in range(fP.I)))+sum(sum(cV.pi[i]*fV.CG[i,:] for i in range(fP.I)))+sum(fV.TH)+sum(fV.TG))
    # absolute target
    # GOVCONSO = np.log(sum(cV.pi[i]*fV.CG[i,:] for i in range(fP.I))) - np.log(sum(fV0.CG[i,:] for i in range(fP.I)))
    for g in range(fP.G):
        equilibrium = np.concatenate((equilibrium,[GOVCONSO[g]]))
    for m in range(fP.M):
        for i in range(fP.I):
            if cP.bl[m,i]==False:
                equilibrium = np.concatenate((equilibrium,[FLOWS(fP, fV_mat)[m,i]]))
    return equilibrium

def equilibriumXGOV(x, fP, cP, fV0, iV0, fV0_mat, fVnew, pVBAU, pVtarget):
    cV = controlVariables(fP)
    mu = x[0] # control variable but outside cV
    tauadjust = x[1] # gov budget adjustment variable, then you select the tax when you affect tauadjust a value
    pV = policyVariables(fP)
    pV.tauX = mu*pVBAU.tauX + (1-mu)*pVtarget.tauX
    for h in range(fP.H-1):
        # pV.tauX[i] = tauadjust
        # pV.tauX[i] = tauadjust*pVBAU.tauX[i] + (1-tauadjust)*pVtarget.tauX[i]
        pV.tauH[h] = tauadjust*pVBAU.tauH[h]
    # pV.tauH = mu*pVBAU.tauH + (1-mu)*pVtarget.tauH
    pV.tauG = mu*pVBAU.tauG + (1-mu)*pVtarget.tauG
    pV.tauR = mu*pVBAU.tauR + (1-mu)*pVtarget.tauR
    pV.tauWI = mu*pVBAU.tauWI + (1-mu)*pVtarget.tauWI
    pV.tauWH = mu*pVBAU.tauWH + (1-mu)*pVtarget.tauWH
    pV.tauWG = mu*pVBAU.tauWG + (1-mu)*pVtarget.tauWG
    cV_mat = controlVariables_mat(fP)
    cV.y[0:fP.I] = x[2:fP.I+2]
    cV.zh[0:fP.H] = x[fP.I+2:fP.I+fP.H+2]
    cV.zg[0:fP.G] = x[fP.I+fP.H+2:fP.I+fP.H+fP.G+2]
    cV.pi[0:fP.I] = x[fP.I+fP.H+fP.G+2:2*fP.I+fP.H+fP.G+2]
    cV.pf[0:fP.F] = x[2*fP.I+fP.H+fP.G+2:2*fP.I+fP.H+fP.G+fP.F+2]
    index = 0
    for m in range(fP.M):
        for i in range(fP.I):
            if cP.bl[m,i]==False:
                cV_mat.lbd[m,i] = x[2*fP.I+fP.H+fP.G+fP.F+2+index]
                index += 1
            else:
                cV_mat.lbd[m,i] = 1.
    return equilibriumGOV(fP, cV, cV_mat, pV, cP, fV0, iV0, fV0_mat, fVnew)

def funcObjectiveXGOV(x): # it is better to call a function than directly the variable x[0]
    return x[0]

def funcEquilibriumGOV(fP, cV0, cV0_mat, cP, fV0, iV0, fV0_mat, fVnew, pVBAU, pVtarget, display=True):
    x = np.full(fP.I*(2+fP.M)+fP.H+fP.G+fP.F+2, 1.)
    x[0] = 1. #(mu starting point is BAU or previous year)
    x[1] = 1. #give the right adjustment tax
    x[2:fP.I+2] = cV0.y[0:fP.I]
    x[fP.I+2:fP.I+fP.H+2] = cV0.zh[0:fP.H]
    x[fP.I+fP.H+2:fP.I+fP.H+fP.G+2] = cV0.zg[0:fP.G]
    x[fP.I+fP.H+fP.G+2:2*fP.I+fP.H+fP.G+2] = cV0.pi[0:fP.I]
    x[2*fP.I+fP.H+fP.G+2:2*fP.I+fP.H+fP.G+fP.F+2] = cV0.pf[0:fP.F]
    for m in range(fP.M):
        for i in range(fP.I):
            if cP.bl[m,i]==False:
                x = np.concatenate((x,[cV0_mat.lbd[m,i]]))
    x_bound = [(0.0000000*x[i],2.*x[i]) for i in range(len(x)-2)]
    x_bound = np.concatenate(([(0.,1.)],[(0.,10.)],x_bound))
    # test if equilibriumX returns zero for mu=1 (if not, it means initial exogenous data wrong)
    # to be adjusted: does not work when non stationary
    # testEquil = equilibriumX(x, fP, cP, fV0, iV0, fV0_mat, fVnew, pVBAU, pVtarget)
    # testEquil = sum(testEquil*testEquil)
    # if testEquil>0.000001:
    #     print('Norm of the equil is', testEquil)
    #     raise TypeError('Issue with the data: initial equilibrium is not found')
    Optimised = minimize(funcObjectiveXGOV, x, args=(),method='SLSQP', bounds = tuple(x_bound), options={'disp': display}, constraints=[{"type":"eq", "fun":equilibriumXGOV,"args":(fP, cP, fV0, iV0, fV0_mat, fVnew, pVBAU, pVtarget)}])
    cV = controlVariables(fP)
    cV_mat = controlVariables_mat(fP)
    mu = Optimised.x[0]
    tauadjust = Optimised.x[1] #must also give new value to pVtarget
    cV.y[0:fP.I] = Optimised.x[2:fP.I+2]
    cV.zh[0:fP.H] = Optimised.x[fP.I+2:fP.I+fP.H+2]
    cV.zg[0:fP.G] = Optimised.x[fP.I+fP.H+2:fP.I+fP.H+fP.G+2]
    cV.pi[0:fP.I] = Optimised.x[fP.I+fP.H+fP.G+2:2*fP.I+fP.H+fP.G+2]
    cV.pf[0:fP.F] = Optimised.x[2*fP.I+fP.H+fP.G+2:2*fP.I+fP.H+fP.G+fP.F+2]
    index = 0
    for m in range(fP.M):
        for i in range(fP.I):
            if  cP.bl[m,i]==False:
                cV_mat.lbd[m,i] = Optimised.x[2*fP.I+fP.H+fP.G+fP.F+2+index]
                index += 1
            else:
                cV_mat.lbd[m,i] = 1.
    return [mu,tauadjust,cV,cV_mat]

# Simulations

## Load data

In [ ]:
# read the baseline

SAMbau = SAM('SAM.xlsx','SAM')
PIOTbau_iron = PIOT('SAM.xlsx','PIOT1',name='iron')
PIOTbau_carbon = PIOT('SAM.xlsx','PIOT2',name='carbon')
# PIOTs_bau = [PIOTbau_iron]
PIOTs_bau = [PIOTbau_iron,PIOTbau_carbon]
fP = fixedParameters(SAMbau,nb_years = 1)
eP = Parameters(fP)
eP.delta = np.array([19448/30194]) # this needs to be changed for BAU
eP.deltam = np.array([19448/30194, 19448/30194]) # this needs to be changed for BAU
# eP.delta = np.array([4/17]) # this needs to be changed for BAU
# eP.deltam = np.array([4/17, 4/17]) # this needs to be changed for BAU
sec = sectors(SAMbau)

fV0 = baseline(SAMbau, fP)
fV0_mat = baseline_mat(PIOTs_bau,SAMbau,fP)
pV0 = funcInitPolicy(fP,'SAM.xlsx','TAX',sec) # Compute the baseline policies

print('baseline policies:')
print('tauXI:', pV0.tauXI[0,:])
print('tauXH:', pV0.tauXH[0,:])
print('tauXG:', pV0.tauXG[0,:])
print('tauI:', pV0.tauI[0,:])
print('tauH:', pV0.tauH[0,:])
print('tauG:', pV0.tauG[0,:])
print('tauRI:', pV0.tauRI[0,:])
print('tauWI:', pV0.tauWI[0,:])
print('tauWH:', pV0.tauWH[0,:])
print('tauWG:', pV0.tauWG[0,:])
print('tauKI:', pV0.tauKI[0,:])
print('tauKH:', pV0.tauKH[0,:])

# print('SAM:\n', SAMbau.data,'\n')
# for m in PIOTs_bau:
#     print('PIOT '+m.name+':\n', m.data,'\n')

[cV0, cV0_mat, iV0] = funcInitControlV(fP, fV0, fV0_mat) # Initiate baseline control variables 

cP = funcCalibrate(fP,iV0,fV0,fV0_mat,pV0,display=True) # Calibrate the variables

sV0_mat = funcInitCapital(fP, fV0_mat, fV0, cP)

print('Initial material capital:', sV0_mat.Km[0,:])

fV = funcSimulate(fP, cP, fV0, pV0, iV0, cV0)
[iV, fV_mat] = funcSimulate_mat(fP, cP, iV0, cV0_mat, fV)

rV = reportVariables(fP,cV0,fV,fV_mat,pV0,iV,cP)

print(EQUILIBRIUM(fP, cP, eP, cV0, cV0_mat, pV0, fV, iV, inflation= 3, display=True))

# return norm in case (to see if there is a big value in the array)
# if the depreciation rate of capital is not calbrated for BAU, it won't return 0
print('squared sum of equil',np.sum(EQUILIBRIUM(fP, cP, eP, cV0, cV0_mat, pV0, fV, iV, display=False)*EQUILIBRIUM(fP, cP, eP, cV0, cV0_mat, pV0, fV, iV, display=False)))


## Scenarios

In [ ]:
#Scens as a dictionary instead of an array
#Each dictionary entry key must be different !!
Scenarios = {}

### Test scenarios

BAU

In [ ]:
# INT BAU

start_time =[]
end_time = []

Scenarios["BAU"] = {}

#######################

SAMbau = SAM('SAM.xlsx','SAM')
PIOTbau_iron = PIOT('SAM.xlsx','PIOT1',name='iron')
PIOTbau_carbon = PIOT('SAM.xlsx','PIOT2',name='carbon')
# PIOTs_bau = [PIOTbau_iron]
PIOTs_bau = [PIOTbau_iron,PIOTbau_carbon]
fP = fixedParameters(SAMbau,nb_years = 1)
eP = Parameters(fP)
eP.delta = np.array([19448.09996/30194.24826]) # this needs to be changed for different BAU
eP.deltam = np.array([19448.09996/30194.24826, 19448.09996/30194.24826]) # this needs to be changed for BAU
# eP.delta = np.array([4/17]) # this needs to be changed for BAU
# eP.deltam = np.array([4/17, 4/17]) # this needs to be changed for BAU
sec = sectors(SAMbau)

fV0 = baseline(SAMbau, fP)
fV0_mat = baseline_mat(PIOTs_bau,SAMbau,fP)
pV0 = funcInitPolicy(fP,'SAM.xlsx','TAX',sec) # Compute the baseline policies
pVBAU = copy.deepcopy(pV0)
pVtarget = copy.deepcopy(pV0)

[cV0, cV0_mat, iV0] = funcInitControlV(fP, fV0, fV0_mat) # Initiate baseline control variables 

cP = funcCalibrate(fP,iV0,fV0,fV0_mat,pV0,display=False) # Calibrate the variables

sV0_mat = funcInitCapital(fP, fV0_mat, fV0, cP)

#######################

[mu,tauadjust,cV,cV_mat] = funcEquilibriumGOV(fP, cV0, cV0_mat, cP, eP, fV0, iV0, pVBAU, pVtarget)
# [mu,cV,cV_mat] = funcEquilibrium(fP, cV0, cV0_mat, cP, eP, fV0, iV0, pVBAU, pVtarget)

print(mu)
print(tauadjust)
# pVtarget.tauXI[:,:,:] = tauadjust[:,None,None]*pVBAU.tauXI[:,:,:]
# pVtarget.tauXH[:,:,:] = tauadjust[:,None,None]*pVBAU.tauXH[:,:,:]
# pVtarget.tauXG[:,:,:] = tauadjust[:,None,None]*pVBAU.tauXG[:,:,:]
# pVtarget.tauH[:,:,:] = tauadjust[:,None,None]*pVBAU.tauH[:,:,:]
pVtarget.tauKI[:,:,:] = tauadjust[:,None,None]*pVBAU.tauKI[:,:,:]

iV = MAT(fP, cV_mat, iV0)
fV = funcSimulate(fP, cP, fV0, pVtarget, iV, cV)

[iV, fV_mat] = funcSimulate_mat(fP, cP, iV0, cV_mat, fV)
[sV_mat,fV_mat] = deltaCapital(fP, eP, sV0_mat, fV_mat, fV)

rV = reportVariables(fP,cV,fV,fV_mat,pVtarget,iV,cP)

print('output: \n', fV.Y)
# print(fV.CH)

print('squared sum of equil',np.sum(EQUILIBRIUM(fP, cP, eP, cV, cV_mat, pVtarget, fV0, iV0, display=True)*EQUILIBRIUM(fP, cP, eP, cV, cV_mat, pVtarget, fV0, iV0, display=False)))

#######################

Scenarios["BAU"]=scenario(fP,name="BAU",pV=pV0,cV=cV,cV_mat=cV_mat,fV=fV,fV_mat=fV_mat,iV=iV,sV_mat=sV_mat,rV=rV)

# modify the total waste in the report variable ?

# adjust export and display functions

# print(writeNewValueSAM(sec, Scenarios["BAU"].fV, Scenarios["BAU"].cV, t = 1))
# print(writeNewPIOT(sec, Scenarios["BAU"].fV_mat, Scenarios["BAU"].cV_mat, 'Iron', t = 1))

# writeLatex(Scenarios["BAU"],sec,"output - BAU","Both",static=True,PIOT=True, t = 1)

# writeExcel(Scenarios["BAU"],sec,"outputDyn - BAU","Both",PIOT=True,static=False,t=0)
